In [1]:
import csv
from collections import defaultdict
import xlrd

def parse_xlsx(f):
    book = xlrd.open_workbook(f)
    assert book.nsheets == 1
    
    sh = book.sheet_by_index(0)
    headers = [sh.cell_value(rowx=0, colx=i) for i in range(sh.ncols)]
    res = {}
    for i in range(sh.ncols):
        res[headers[i]] = [sh.cell_value(rowx=j, colx=i)  for j in range(1, sh.nrows)]
    
    return res

def parse_csv(f):
    data = defaultdict(list)
    with open("/home/louis/code/hackqc2018/map/data/montrealnordarbrespublics20161207.csv") as fp:
        reader = csv.reader(fp)
        headers = next(reader)
        for row in reader:
            for h, c in zip(headers, row):
                data[h].append(c)

    return dict(data)
#         print(row)

In [90]:
import os
from tqdm import tqdm
path = "/home/louis/code/hackqc2018/map/data/trees"
datas = {}
for file in tqdm(os.listdir(path)):
    try:
        file_path = "{}/{}".format(path, file)
        if file.endswith('.csv'):
            datas[file] = parse_csv(file_path)
        elif file.endswith('.xlsx'):
            datas[file] = parse_xlsx(file_path)
        else:
            raise ValueError(file)
    except Exception as e:
        print("Error on {}".format(file))
        print(e.__repr__())
    

 19%|█▉        | 3/16 [00:05<00:25,  1.97s/it]

Error on etatinventairearbrespublicspararrondissement.xlsx
XLRDError("Unsupported format, or corrupt file: Expected BOF record; found b'Num\\xc3\\xa9ro '",)


100%|██████████| 16/16 [00:56<00:00,  3.51s/it]


In [91]:
from itertools import chain

keys = [list(d) for d in datas.values()][0]
res = {}

for k in tqdm(keys):
    res[k] = [e for d in datas.values() for e in d[k] ]



100%|██████████| 24/24 [00:00<00:00, 88.06it/s]


In [92]:
headers = list(res)

rows = [row for row in zip(*res.values()) if row[8] and row[9]]


In [22]:
set(res['Propriété'])


{'',
 '2002-05-30',
 '2002-06-12',
 '2002-09-06',
 '2004-08-19',
 '2004-12-01',
 '2005-08-11',
 '2012-09-25',
 'A déterminer',
 'Copropriété',
 'Mitoyen',
 'Privé',
 'Ville'}

In [112]:
from subprocess import Popen, PIPE
converter_js = "/home/louis/code/hackqc2018/Geographic-MTM-to-LatLon-Converter/src/geographic-mtm-converter.js"

def convert(args):
    """ args is a list of triplet (w, h, zone)
        return a list of (lat,long)
    """
    payload = "".join("{}, {}, {}\n".format(*a) for a in args).encode('utf8')

    with Popen(["node", converter_js], stdout=PIPE, stdin=PIPE) as proc:
        output = proc.communicate(input=payload)[0].decode('utf8').strip()
        return [[float(e) for e in line.split(',')] for line in output.split('\n')]



SyntaxError: unexpected EOF while parsing (<ipython-input-112-5042dd02c8fe>, line 10)

In [103]:
from pymongo import MongoClient, GEO2D
import utm

db = MongoClient()['hackqc2018']
db.trees.create_index([("loc", GEO2D)])

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


for chk_rows in tqdm(chunks(rows, 1000), total=len(rows)/1000):
    latlons = convert([(float(row[8]), float(row[9]), 8) for row in chk_rows])
    db.trees.insert_many([{
        'loc': latlon,
        **dict(zip(headers, row))
    } for row, latlon in zip(chk_rows, latlons)])


292it [00:53,  5.44it/s]                             


In [116]:
from geojson import Feature, Point, FeatureCollection, dumps



cursor = db.trees.find({'loc': {'$geoWithin': {
    '$box': [
          [45.5166326,-73.6007412],
          [45.524072, -73.582893]
        ]
}}})

fc = FeatureCollection([Feature(geometry=Point(doc['loc'][::-1]), properties={
    'type': doc['Essence_Français'],
#     'age': doc['Date_plant']
}) for doc in tqdm(cursor)])


with open("/home/louis/code/hackqc2018/map/src/trees.js", 'w') as fp:
    fp.write("""module.exports = (()=>{
        return %s
    })()"""%dumps(fc))
    

1760it [00:00, 25984.54it/s]


In [43]:
TESTS = {
    (294551.328 ,   5044698.908): (45.54212936363354, -73.63123853240397),
    (296354.436, 5046722.007):(45.56035808507587, -73.60818394603155),
    (296352.045, 5046719.009):(45.56033107909284, -73.608214521844)
}

In [68]:
for mtm, latlon in TESTS.items():
    r = convert(*mtm, 8)
    

0.0 0.0
0.0 0.0
0.0 0.0


In [118]:
 from owslib.wms import WebMapService
wms = WebMapService('http://wms.jpl.nasa.gov/wms.cgi', version='1.1.1')
